<a href="https://colab.research.google.com/github/pagfarias/exercicios-tera/blob/main/Desafio_de_Infer%C3%AAncia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio de Inferência


> **Transtorno depressivo:**
Problema de saúde pública (prevalência de ≈ 10%)
Multifatorial (fatores sociais, psicológicos, biológicos)


> **Estilo de vida saudável (OMS):**
Prática regular de atividade física
Alimentação saudável

---

#Os Dados

> Resultados da pesquisa NHNES (National Health and Nutrition Examination Survey), realizada anualmente nos EUA para avaliar a **saúde e nutrição** de adultos e crianças. Inclui dados demográficos, socioeconômicos, dietéticos e 
relacionados à saúde
Período de 2005-2006

> **DEMO_PHQ.csv** = Dados **demográficos** e resultados do Patient Health Questionnaire-9 (PHQ-9), usado para avaliar o grau de **sintomas depressivos**.
Inclui apenas adultos.

> **PAG_HEI.csv** = Dados referentes à **atividade física** e ao Healthy Eating Index (HEI), um questionário que avalia a **qualidade da dieta**. Inclui adultos e crianças.

---
#Perguntas a serem respondidas

1. Qual o **perfil** de indivíduos (adultos maiores de 18 anos) com sintomas depressivos nos EUA no período de 2005-2006?
2. Hábitos saudáveis de alimentação e atividade física estão **associados** a menores índices de depressão nesta população?


# Import de Bibliotecas e Bases

In [110]:
#import de bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
#import bases
df_demo_phq = pd.read_csv('DEMO_PHQ.csv')
df_pag_hei = pd.read_csv('PAG_HEI.csv')

# Tratamento dos Dados

In [103]:
#concatenando as duas bases usando como chave a variável SEQN
#df = pd.merge(df_demo_phq, df_pag_hei, on="SEQN")
df = df_demo_phq.merge(df_pag_hei, on = 'SEQN', how = 'left')

In [104]:
#df_demo_phq
#df_pag_hei
df.info()
#df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5334 entries, 0 to 5333
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   SEQN                      5334 non-null   int64  
 1   DPQ010                    4836 non-null   float64
 2   DPQ020                    4836 non-null   float64
 3   DPQ030                    4836 non-null   float64
 4   DPQ040                    4836 non-null   float64
 5   DPQ050                    4836 non-null   float64
 6   DPQ060                    4836 non-null   float64
 7   DPQ070                    4836 non-null   float64
 8   DPQ080                    4836 non-null   float64
 9   DPQ090                    4836 non-null   float64
 10  RIAGENDR                  5334 non-null   int64  
 11  RIDAGEYR                  5334 non-null   int64  
 12  RIDRETH1                  5334 non-null   int64  
 13  DMDEDUC                   5334 non-null   int64  
 14  INDFMINC

In [ ]:
df.describe()

In [119]:
# Tratar as respostas 7 e 9 como valores ausentes ou nulos
# 7 = Se recusou a responder
# 9 = Não sabe

df['DPQ010'] = df['DPQ010'].replace([7,9],np.nan)
df['DPQ020'] = df['DPQ020'].replace([7,9],np.nan)
df['DPQ030'] = df['DPQ030'].replace([7,9],np.nan)
df['DPQ040'] = df['DPQ040'].replace([7,9],np.nan)
df['DPQ050'] = df['DPQ050'].replace([7,9],np.nan)
df['DPQ060'] = df['DPQ060'].replace([7,9],np.nan)
df['DPQ070'] = df['DPQ070'].replace([7,9],np.nan)
df['DPQ080'] = df['DPQ080'].replace([7,9],np.nan)
df['DPQ090'] = df['DPQ090'].replace([7,9],np.nan)

In [120]:
#Criando a variável phq9
df['PHQ9'] = df['DPQ010'] + df['DPQ020'] + df['DPQ030'] + df['DPQ040'] + df['DPQ050'] + df['DPQ060'] + df['DPQ070'] + df['DPQ080'] + df['DPQ090'] 

In [121]:
df['PHQ9'].describe()

count    4799.000000
mean        2.732236
std         3.727676
min         0.000000
25%         0.000000
50%         1.000000
75%         4.000000
max        27.000000
Name: PHQ9, dtype: float64

In [122]:
df['PHQ9'].unique()

array([nan,  0.,  4.,  6.,  1.,  3.,  2.,  8., 12.,  5.,  7., 14., 10.,
       13., 18.,  9., 16., 15., 11., 17., 19., 20., 21., 23., 22., 24.,
       27., 25., 26.])

In [123]:
#Crianado a varial de agrupamento de PHQ
#Crie a variável phq_grp com a classificação do escore phq9, assumindo os valores:
#● 0 (“sem sintomas”) se phq9 < 5
#● 1 (“sintomas leves”) se 5 <= phq9 < 10
#● 2 (“sintomas moderados”) se 10 <= phq9 < 15
#● 3 (“sintomas moderadamente severos”) se 15 <= phq9 < 19
#● 4 (“sintomas severos”) se phq9 >= 20

criterio = [
    (df['PHQ9'].isna()),
    (df['PHQ9'] <= 5),
    (df['PHQ9'] > 5) & (df['PHQ9'] <= 9),
    (df['PHQ9'] > 9) & (df['PHQ9'] <= 14),
    (df['PHQ9'] > 14) & (df['PHQ9'] <= 19),
    (df['PHQ9'] > 19)    
]

grupo = [np.nan,0,1,2,3,4]

df['PHQ_GRP'] = np.select(criterio,grupo)

In [124]:
df
df[['PHQ_GRP']].value_counts(sort = False)

PHQ_GRP
0.0        4013
1.0         489
2.0         201
3.0          73
4.0          23
dtype: int64

In [125]:
#Como as categorias 2,3 e 4 há poucas pessoas, vamos agrupá-las

df['PHQ_GRP'] = df['PHQ_GRP'].replace([3,4],2)
df[['PHQ_GRP']].value_counts(sort = False)

PHQ_GRP
0.0        4013
1.0         489
2.0         297
dtype: int64